In [1]:
import json
import pandas as pd
from tqdm import tqdm_notebook
import MySQLdb
import re
from time import sleep
import os

In [2]:
from datetime import date, timedelta

sdate = date(2019,11, 1)   # start date
edate = date(2019, 11, 30)   # end date

delta = edate - sdate       # as timedelta

date_list = []
for i in range(delta.days + 1):
    day = (sdate + timedelta(days=i)).isoformat()
    date_list.append(day)
print(date_list)

['2019-11-01', '2019-11-02', '2019-11-03', '2019-11-04', '2019-11-05', '2019-11-06', '2019-11-07', '2019-11-08', '2019-11-09', '2019-11-10', '2019-11-11', '2019-11-12', '2019-11-13', '2019-11-14', '2019-11-15', '2019-11-16', '2019-11-17', '2019-11-18', '2019-11-19', '2019-11-20', '2019-11-21', '2019-11-22', '2019-11-23', '2019-11-24', '2019-11-25', '2019-11-26', '2019-11-27', '2019-11-28', '2019-11-29', '2019-11-30']


In [3]:
arr = os.listdir('./Drag_Data/')
file_list =[]
for _ in arr:
    if _.split('_')[-1].split('.')[0] in date_list:
        #print(_)
        file_list.append(_)


In [4]:
file_list = sorted(file_list)

In [6]:
def outerjoin(left_df, right_df, right_df_new_index_name= 'GameDrivenIndex', left_df_new_index_name=None):
    import pandas as pd
    if left_df_new_index_name is not None:
        left_df = left_df.rename(columns={'index':left_df_new_index_name})
    right_df = right_df.rename(columns={'index':right_df_new_index_name})
    left_df['dummy']= 'HH'
    right_df['dummy'] = 'HH'
    result = pd.merge(left_df, right_df, on='dummy', how='left')
    return result.loc[:, result.columns != 'dummy']

In [7]:
connection = MySQLdb.connect(host='103.22.220.149',port = 13306, user='jaeil', password='yonsei2019!',
                             database = 'Project_Dobrain', use_unicode =True, charset = 'utf8')
print('connection success')
cursor = connection.cursor()
print('generate cursor')
SQL = 'INSERT INTO Drag_Data_by_Date_JSON' + ' VALUES (%s)' % str('%s,'*20 + '%s')
final_real=pd.DataFrame()
for _ in tqdm_notebook(file_list):
    print(_)
    json_data=open('./Drag_Data/'+_).read()
    data = json.loads(json_data)

    for id_ in tqdm_notebook(range(len(data))):
        final=pd.DataFrame()

        data_temp = data[id_]
        temp_ = pd.DataFrame.from_dict(data_temp)

        left = temp_.iloc[:-1]
        right = temp_.iloc[-1]

        left=left.drop_duplicates()
        left = left.T
        left.reset_index(inplace=True)
        left.rename(columns={'index':'ID'}, inplace=True)

        
        for level in range(len(right)):
                try:
                    right1 = pd.DataFrame.from_dict(right.values[level]).iloc[:,-1]
                    left1 = pd.DataFrame.from_dict(right.values[level]).iloc[:,:-1]
                    left1=left1.drop_duplicates()

                except IndexError:
                    print("There is no id in the date")
                    continue
                for h in right1:

                    left2 = pd.DataFrame.from_dict(h).iloc[:,:-1]
                    right2 = pd.DataFrame.from_dict(h).iloc[:,-1]

                    left2=left2.drop_duplicates()
                    #
                    for i in right2:
  
                        left3 = pd.DataFrame.from_dict(i).iloc[:,:-1]
                        right3 = pd.DataFrame.from_dict(i).iloc[:,-1]
                        left3=left3.drop_duplicates()

                        for j in range(len(right3)):
                            
                            left4 = pd.DataFrame.from_dict(right3.values[j]).iloc[:,:-1]
                            right4 = pd.DataFrame.from_dict(right3.values[j]).iloc[:,-1]
                            left4=left4.drop_duplicates()
           
                            for k in right4:
                                try:
                                    left5 = pd.DataFrame.from_dict(k).iloc[:,:-1]
                                    right5 = pd.DataFrame.from_dict(k).iloc[:,-1]
                                    temp=pd.DataFrame()
     

                                except ValueError:
                                    continue
                                for m in right5:
                                    try:
                                        if str(type(m))=="<class 'list'>":
                                            left6 = pd.DataFrame.from_dict(m)
                                        else:
                                            left6 = pd.DataFrame.from_dict(list(m))
                                        #left6 = pd.DataFrame.from_dict(left6)
                                        left6.rename(columns={'creationDateTime':'creationDateTime_m'}, inplace=True)
                                        temp=temp.append(left6, ignore_index=True)
                                        #print(left6)
                                    except:
                                        continue
                                output1=outerjoin(left5,temp)

                                output2=outerjoin(left4,output1)
                                #print(output2)
                                output3 = outerjoin(pd.DataFrame(left3.iloc[j]).T, output2)
                         
                               #print('output3 : ', output3)

                                output4=outerjoin(left2, output3)
   
                                output5=outerjoin(left1, output4)

                                output6=outerjoin(left, output5)

                                output6['Date']=_.split('_')[-1].split('.')[0]
            
                                final=final.append(output6, ignore_index=True)

        final.rename(columns={'_deviceModel':'DeviceModel','_deviceName':'DeviceName','_dpi':'Dpi','_screenHeight':'ScreenHeight','_screenWidth':'ScreenWidth'
                                                      ,'_level':'Level','_index':'Index_','_index_x':'Index_X','_questionManagerCategory':'QuestionManagerCategory','_index_y':'Index_Y'
                                                      ,'appVersion':'AppVersion','creationDateTime':'CreationDatetime','category':'Category','creationDateTime_m':'CreationDatetime_M'
                                                      ,'GameDrivenIndex':'GameGrivenIndex','isOnCorrectAnswer':'IsOncorrectAnswer','posX':'PosX','posY':'PosY','touchPressure':'TouchPressure','Date':'Date','ID':'ID'}, inplace=True)
        for col in ['Date','ID','DeviceModel','DeviceName','Dpi','ScreenHeight','ScreenWidth','Level','Index_','Index_X','QuestionManagerCategory',
                     'Index_Y','AppVersion','CreationDatetime','Category','CreationDatetime_M','GameGrivenIndex','IsOncorrectAnswer','PosX','PosY','TouchPressure']:
            if col not in final.columns:
                final[col]=0
            else:
                continue
        #print(type(output6))
        final=final[['Date','ID','DeviceModel','DeviceName','Dpi','ScreenHeight','ScreenWidth','Level','Index_','Index_X','QuestionManagerCategory',
                 'Index_Y','AppVersion','CreationDatetime','Category','CreationDatetime_M','GameGrivenIndex','IsOncorrectAnswer','PosX','PosY','TouchPressure']]
        for num in range(len(final)):
            try:
                cursor.execute(SQL, final.iloc[num,:])
            except:
                print('Excute Fail')
                print(final.iloc[num,:])
                break
        connection.commit()
        #print("Excute sucessed")
    #final_real=final_real.append(final, ignore_index=True)
cursor.close()
connection.close()

connection success
generate cursor


drag_data_by_date_2019-11-01.json


/home/jaeil/.local/lib/python3.6/site-packages/pandas/core/frame.py:7138: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  sort=sort,


Excute Fail
Date                                             2019-11-01
ID                         0a65c33141f7cbb6ca22bf97fc77f423
DeviceModel                             Xiaomi Redmi Note 5
DeviceName                                               진기
Dpi                                                     440
ScreenHeight                                           1080
ScreenWidth                                            2160
Level                                                     A
Index_                                                    1
Index_X                                                   7
QuestionManagerCategory                     DrawingQManager
Index_Y                                                   0
AppVersion                                           3.35.0
CreationDatetime                        2019-11-01 08:42:24
Category                                                NaN
CreationDatetime_M                                      NaN
GameGrivenIndex             

Excute Fail
Date                                             2019-11-02
ID                         6c51e7033a7849e8e651767090dac949
DeviceModel                                 HUAWEI AGS2-W09
DeviceName                               HUAWEI MediaPad T5
Dpi                                                     320
ScreenHeight                                           1200
ScreenWidth                                            1920
Level                                                     C
Index_                                                   25
Index_X                                                   2
QuestionManagerCategory                     DrawingQManager
Index_Y                                                   0
AppVersion                                           3.35.0
CreationDatetime                        2019-11-02 01:20:57
Category                                                NaN
CreationDatetime_M                                      NaN
GameGrivenIndex             

...

Excute Fail
Date                                             2019-11-26
ID                         0ad923e7f23efa80e37c63041d76ae71
DeviceModel                                 samsung SM-T280
DeviceName                              Galaxy Tab A (2016)
Dpi                                                     213
ScreenHeight                                            800
ScreenWidth                                            1280
Level                                                     A
Index_                                                    7
Index_X                                                   4
QuestionManagerCategory                     DrawingQManager
Index_Y                                                   0
AppVersion                                           3.35.0
CreationDatetime                        2019-11-26 12:17:09
Category                                                NaN
CreationDatetime_M                                      NaN
GameGrivenIndex             

Excute Fail
Date                                                 2019-11-27
ID                         823D6074-CF07-41A9-8915-85A3CB32DBA2
DeviceModel                                          iPhone11,8
DeviceName                                              🖤seula🌙
Dpi                                                         326
ScreenHeight                                                828
ScreenWidth                                                1792
Level                                                         A
Index_                                                       15
Index_X                                                       1
QuestionManagerCategory                     DragAndDropQManager
Index_Y                                                       0
AppVersion                                               3.35.0
CreationDatetime                            2019-11-27 10:15:02
Category                                                     -1
CreationDatetime_M          

Excute Fail
Date                                             2019-11-28
ID                         05b159af24f8f55b1e27c047e0dd95d6
DeviceModel                                samsung SM-N950N
DeviceName                                     Galaxy Note8
Dpi                                                     420
ScreenHeight                                           1080
ScreenWidth                                            2220
Level                                                     A
Index_                                                    9
Index_X                                                   2
QuestionManagerCategory                     DrawingQManager
Index_Y                                                   0
AppVersion                                           3.35.0
CreationDatetime                        2019-11-28 09:01:41
Category                                                NaN
CreationDatetime_M                                      NaN
GameGrivenIndex             

Excute Fail
Date                                                 2019-11-29
ID                         150F6166-62A0-487C-ADC4-A660CBD345D4
DeviceModel                                           iPhone8,1
DeviceName                                               iPhone
Dpi                                                         326
ScreenHeight                                                750
ScreenWidth                                                1334
Level                                                         A
Index_                                                        9
Index_X                                                       4
QuestionManagerCategory                         DrawingQManager
Index_Y                                                       0
AppVersion                                               3.35.0
CreationDatetime                            2019-11-29 13:00:03
Category                                                    NaN
CreationDatetime_M          

Excute Fail
Date                                             2019-11-30
ID                         0c763944d99caf7f950d1b3e392d86cc
DeviceModel                                samsung SM-N960N
DeviceName                                     Galaxy Note9
Dpi                                                     420
ScreenHeight                                           1080
ScreenWidth                                            2220
Level                                                     A
Index_                                                    5
Index_X                                                   6
QuestionManagerCategory                     DrawingQManager
Index_Y                                                   0
AppVersion                                           3.35.0
CreationDatetime                        2019-11-30 10:01:50
Category                                                NaN
CreationDatetime_M                                      NaN
GameGrivenIndex             